In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import random

In [4]:
def rand_walk(seed, steps):
    #mutes audio
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--mute-audio")

    driver = webdriver.Chrome(options=chrome_options)
    result = []
    for i in range(steps+1):
        driver.get(seed)
        driver.maximize_window()
        time.sleep(5)    

        #Gathers the future links
        thumbnails = driver.find_elements_by_id("thumbnail")
        links = [thumb.get_attribute("href") for thumb in thumbnails]
        links.pop()
        
        #gathers the current video
        curr_vid = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
        curr_channel = driver.find_element_by_xpath('//*[@id="text"]/a').text

        
        
        #checks if link is empty
        #if it is, we keep the same link and retry
        try:
            if not links == None and not len(links) == 0:
                #appends the results if the next link is valid

                #makes sure link is valid
                #randomly chooses a video from the top 10 videos
                if len(links) >= 9:
                    choice = random.choice(range(0,9))
                else:
                    choice = random.choice(range(0,len(links)-1))
                newlink = links[choice]

                if newlink == None:
                    i = i -1
                else:
                    while not 'https://www.youtube.com/watch' == newlink[0:29]:
                        print('https://www.youtube.com/watch' == newlink[0:29])
                        if len(links) >= 9:
                            choice = random.choice(range(0,9))
                        else:
                            choice = random.choice(range(0,len(links)-1))
                        newlink = links[choice]
                        if newlink == None:
                            i = i -1
                            break

                    result.append(curr_channel)
                    result.append(curr_vid)
                    result.append(len(links))
                    seed = newlink
            else:
                i = i - 1
        except:
            i = i - 1


        
    driver.quit()
    return result

In [12]:
def channel_scraper(url, numvideos):
    data = []
    cols = ["Channel","Subscribers", "Video Name", "Views", "Link"]

    #mutes audio
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--mute-audio")
    try:
        #sets up driver and goes into video tab
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        #video_button = driver.find_element_by_xpath('//*[@id="tabsContent"]/paper-tab[2]/div')
        video_button = WebDriverWait(driver,7).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="tabsContent"]/paper-tab[2]/div')))
        video_button.click()
        time.sleep(3)

        #data collections
        channel = driver.find_element_by_xpath('//*[@id="text-container"]').text
        subs = driver.find_element_by_xpath('//*[@id="subscriber-count"]').text
        videos = driver.find_elements_by_class_name("yt-simple-endpoint style-scope ytd-grid-video-renderer")
        vidnames = driver.find_elements_by_xpath('//*[@id="video-title"]')
        views = driver.find_elements_by_xpath('//*[@id="metadata-line"]/span[1]')
        thumbnails = driver.find_elements_by_id("video-title")
        links = [thumb.get_attribute('href') for thumb in thumbnails]
    except:
        driver.close()
        return None
    
    if len(vidnames) == 0:
        driver.close()
        return None

    #gathers the top 100 most recent videos
    n = 100000
    counter = 1
    while len(vidnames) < numvideos:
        script = "window.scrollTo(0," + str(n) +")"
        driver.execute_script(script) 
        n *= 2
        #timer.sleep(2)
        vidnames = driver.find_elements_by_xpath('//*[@id="video-title"]')
        counter += 1
        if counter >= 100:
            break
    #ensures data is uniform
    while(not (len(vidnames) == len(views) == len(links))):
        vidnames = driver.find_elements_by_xpath('//*[@id="video-title"]')
        views = driver.find_elements_by_xpath('//*[@id="metadata-line"]/span[1]')
        thumbnails = driver.find_elements_by_id("video-title")
        links = [thumb.get_attribute('href') for thumb in thumbnails]
    
    #reformating data
    #subscriber count to int
    if 'M subscribers' in subs:
        subs = int(float(subs.replace("M subscribers",""))*1000000)
    elif 'K subscribers' in subs:
        subs = int(float(subs.replace("K subscribers",""))*1000)
    
    views = [view.text for view in views]
    for i in range(len(views)):
        if 'B views' in views[i]:
            views[i] = int(float(views[i].replace('B views','')) * 1000000000)
        elif 'M views' in views[i]:
            views[i] = int(float(views[i].replace('M views','')) * 1000000)
        elif 'K views' in views[i]:
            views[i] = int(float(views[i].replace('K views','')) * 1000)
        else:
            views[i] = 0
    
    #formats into dataframe
    if len(vidnames) == len(views) == len(links):
        for i in range(len(vidnames)):
            #if views[i] >= 1000:
            data.append((channel, subs, vidnames[i].text, views[i], links[i]))
    #returns resulting data frame
    driver.close()
    result = pd.DataFrame(data, columns = cols)
    return result


In [18]:
def vidsim(startvid,steps, num_sims):
    #sets up appropriate amounts of columns
    cols = ["Origin Channel","Origin Video", "recommendation #"]
    data = []
    for i in range(steps):
        cols.append("Step: " + str(i+1) + " Channel")
        cols.append("Step: " + str(i+1) + " Video")
        cols.append("vid " + str(i+1) + " recommendations #")
   
    #gets the data from the video scraper
    for i in range(num_sims):
        scrape = rand_walk(startvid,steps)
        scrape = tuple(scrape)
        data.append(scrape)
    #imports data into df
    df1 = pd.DataFrame(data, columns = cols)
    #return data
    return df1
    
    

In [21]:
vidlist= ['https://www.youtube.com/watch?v=V6Q_nlSULio']
num_sim = 5
steps = 5
df = pd.DataFrame()
for vid in vidlist:
    df = df.append(vidsim(vid,steps,num_sim), ignore_index = True)

In [8]:
tags = pd.read_csv("Hard Tagged Channels.csv")
news = tags[tags["TAGS"].str.contains("Mainstream News")]["CHANNEL_ID"].tolist()
news = ['https://www.youtube.com/channel/' + user for user in news]

data = random.sample(news,10)
channelvids = pd.DataFrame()
    
for channel in news:
    channelvids = channelvids.append(channel_scraper(channel,100))


118
118
120
120
17
120
117
120
90
7
118
118
120
118
120
120
120
120
120
115
1
120
120
108
117
1
120
118
118
120
120
120
6
120
119
120
120
1
119
120
120


In [22]:
df

,Origin Channel,Origin Video,recommendation #,Step: 1 Channel,Step: 1 Video,vid 1 recommendations #,Step: 2 Channel,Step: 2 Video,vid 2 recommendations #,Step: 3 Channel,Step: 3 Video,vid 3 recommendations #,Step: 4 Channel,Step: 4 Video,vid 4 recommendations #,Step: 5 Channel,Step: 5 Video,vid 5 recommendations #
0,Community,The Cast of Community Reunites for Table Read ...,20,The Darkest Timeline with Ken Jeong & Joel McHale,Episode #9 - 8 Podcasts and a Zoom Chat,20,Community,Season 2 Bloopers! #2 | Community,22,Leo Ashe,Best of Troy and Abed: Community S01 - LeoAshe...,22,Comedy Bites,Dwight K. Schrute: ENTREPRENEUR | The Office U...,22,None,None,NaN
1,Community,The Cast of Community Reunites for Table Read ...,20,Community,Continue watching! Cast of Community Q&A follo...,22,Community,Season 2 Bloopers! #2 | Community,22,Comedy Bites,Season 6 BLOOPERS | The Office US | Comedy Bites,22,The Office,Spontaneous Pranks That Drove Dwight Insane - ...,22,None,None,NaN
2,Community,The Cast of Community Reunites for Table Read ...,20,Community,For A Few Paintballs More - Behind The Scenes ...,22,Vulture,In Conversation with the Cast of Community,20,Community,The Cast of Community Reunites for Table Read ...,20,Community,Community Writers' Favourite Moments | Community,22,None,None,NaN
3,Community,The Cast of Community Reunites for Table Read ...,20,Josh Gad,One Zoom to Rule Them All | Reunited Apart LOR...,20,Josh Gad,It's Time to go BACK TO THE FUTURE! | Reunited...,20,Josh Gad,The Goonies Are Back!! | Reunited Apart with J...,20,Josh Gad,It's Time to go BACK TO THE FUTURE! | Reunited...,20,Late Night with Seth Meyers,Mark Hamill Does a Perfect Impression of Harri...,20.0
4,Community,The Cast of Community Reunites for Table Read ...,20,Vulture,In Conversation with the Cast of Community,20,OSSA,Community: What Happened To The Cast? | ⭐OSSA,20,StoryStreet,What Makes Community Great: The Heart Within t...,20,tylermsc,Why Advanced Dungeons and Dragons is the Best ...,20,The Normies,Community - 2x14 Advanced Dungeons and Dragons...,20.0
